In [61]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [62]:
df = pd.read_csv('vowel.dat',header=None,sep = ',')
train = df[df[0] == 0]
test = df[df[0] == 1]

X = train.iloc[:,3:13].to_numpy()
y = train.iloc[:,-1:].to_numpy().reshape(528)
testX = test.iloc[:,3:13].to_numpy()
testy = test.iloc[:,-1:].to_numpy().reshape(462)

numbers = [*range(11)] # number 2 > number 1
number_of_classification = 11


list_y = y.tolist()
digits_indexes = []
for digit in numbers:
    li = [i for i in range(len(list_y)) if list_y[i] == digit]
    digits_indexes.extend(li)

samplesize = X[digits_indexes].shape[0]
x_inputs =[tuple(c) for c in X[digits_indexes].tolist()]
x_outputs = [tuple(c) for c in y[digits_indexes].reshape(samplesize,1).tolist()]

test_list_y =testy.tolist()
digits_indexes = []
for digit in numbers:
    li = [i for i in range(len(test_list_y)) if test_list_y[i] == digit]
    digits_indexes.extend(li)

test_x_inputs =[tuple(c) for c in testX[digits_indexes].tolist()]
test_x_outputs = [tuple(c) for c in testy[digits_indexes].reshape(462,1).tolist()]

In [65]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        
        outputs = []
        for xi in x_inputs:
            output = net.activate(xi)
            outputs.append(output)
        
        
        px_outputs = softmax(np.array(outputs).reshape(samplesize, number_of_classification), axis=1)
        # the index of maximum in each line
        pred_outputs = np.argmax(px_outputs, axis = 1)
        # replace index to real number, from higher number to lower number!!
        for i in range(number_of_classification):
            index = number_of_classification - i - 1
            pred_outputs[pred_outputs==index] = numbers[index]
        
        real_outputs = np.array(x_outputs).reshape(samplesize,)
        
        acc = np.sum(pred_outputs == real_outputs)/samplesize

        genome.fitness = acc

In [68]:
def run(config_file):
    # Load configuration.
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        
        config_file,
    )

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # add a stdout reporter to show progress in the terminal
    reporter = neat.StdOutReporter(False)
    p.add_reporter(reporter)
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #checkpointer = neat.Checkpointer(100)
    #p.add_reporter(checkpointer)
    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 500)

    return [stats, winner]


samplesize = len(x_outputs)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-vowel11")
[stats, winner] = run(config_path)

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)


 ****** Running generation 0 ****** 

Population's average fitness: 0.09021 stdev: 0.02703
Best fitness: 0.18561 - size: (11, 11) - species 6 - id 43
Average adjusted fitness: 0.082
Mean genetic distance 2.655, standard deviation 0.290
Population of 201 members in 24 species
Total extinctions: 0
Generation time: 2.081 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.10387 stdev: 0.02765
Best fitness: 0.18561 - size: (11, 11) - species 6 - id 43
Average adjusted fitness: 0.078
Mean genetic distance 2.769, standard deviation 0.350
Population of 199 members in 27 species
Total extinctions: 0
Generation time: 2.875 sec (2.478 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.11124 stdev: 0.03176
Best fitness: 0.18561 - size: (11, 11) - species 6 - id 43
Average adjusted fitness: 0.078
Mean genetic distance 2.816, standard deviation 0.353
Population of 200 members in 28 species
Total extinctions: 0
Generation time: 3.328 sec (2.761 av

Population's average fitness: 0.13190 stdev: 0.04333
Best fitness: 0.24432 - size: (13, 15) - species 22 - id 2482
Average adjusted fitness: 0.088
Mean genetic distance 3.028, standard deviation 0.496
Population of 202 members in 18 species
Total extinctions: 0
Generation time: 3.202 sec (3.510 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.13053 stdev: 0.04135
Best fitness: 0.24432 - size: (13, 15) - species 22 - id 2482
Average adjusted fitness: 0.086
Mean genetic distance 3.008, standard deviation 0.484
Population of 194 members in 18 species
Total extinctions: 0
Generation time: 4.206 sec (3.654 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.13276 stdev: 0.04204
Best fitness: 0.24432 - size: (13, 15) - species 22 - id 2482
Average adjusted fitness: 0.086
Mean genetic distance 3.052, standard deviation 0.489
Population of 194 members in 17 species
Total extinctions: 0
Generation time: 3.737 sec (3.740 average)

 ***

Population's average fitness: 0.13803 stdev: 0.04011
Best fitness: 0.25189 - size: (13, 16) - species 22 - id 7276
Average adjusted fitness: 0.088
Mean genetic distance 2.908, standard deviation 0.487
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 3.382 sec (3.744 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.13567 stdev: 0.04214
Best fitness: 0.25189 - size: (13, 16) - species 22 - id 7276
Average adjusted fitness: 0.107
Mean genetic distance 2.904, standard deviation 0.480
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 3.440 sec (3.770 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.14117 stdev: 0.04136
Best fitness: 0.25189 - size: (13, 16) - species 22 - id 7276
Average adjusted fitness: 0.112
Mean genetic distance 2.902, standard deviation 0.476
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 3.431 sec (3.806 average)

 ***

Population's average fitness: 0.15418 stdev: 0.04266
Best fitness: 0.26705 - size: (25, 35) - species 37 - id 11778
Average adjusted fitness: 0.092
Mean genetic distance 2.966, standard deviation 0.554
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 5.544 sec (3.936 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.14843 stdev: 0.04464
Best fitness: 0.26705 - size: (25, 35) - species 37 - id 11778
Average adjusted fitness: 0.092
Mean genetic distance 2.985, standard deviation 0.532
Population of 197 members in 14 species
Total extinctions: 0
Generation time: 4.571 sec (4.043 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.14856 stdev: 0.04681
Best fitness: 0.26705 - size: (25, 35) - species 37 - id 11778
Average adjusted fitness: 0.109
Mean genetic distance 3.007, standard deviation 0.522
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 4.390 sec (4.139 average)

 

Population's average fitness: 0.14840 stdev: 0.04487
Best fitness: 0.26894 - size: (25, 37) - species 37 - id 15971
Average adjusted fitness: 0.096
Mean genetic distance 2.803, standard deviation 0.557
Population of 200 members in 8 species
Total extinctions: 0
Generation time: 4.176 sec (4.357 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.14437 stdev: 0.04591
Best fitness: 0.26894 - size: (25, 37) - species 37 - id 15971
Average adjusted fitness: 0.088
Mean genetic distance 2.831, standard deviation 0.536
Population of 199 members in 9 species
Total extinctions: 0
Generation time: 4.191 sec (4.240 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.14441 stdev: 0.04481
Best fitness: 0.26894 - size: (25, 37) - species 37 - id 15971
Average adjusted fitness: 0.070
Mean genetic distance 2.852, standard deviation 0.536
Population of 199 members in 9 species
Total extinctions: 0
Generation time: 4.110 sec (4.157 average)

 ***

...

Population's average fitness: 0.17147 stdev: 0.05852
Best fitness: 0.30114 - size: (30, 50) - species 156 - id 64592
Average adjusted fitness: 0.119
Mean genetic distance 2.944, standard deviation 0.503
Population of 199 members in 13 species
Total extinctions: 0
Generation time: 5.153 sec (4.571 average)

 ****** Running generation 385 ****** 

Population's average fitness: 0.17805 stdev: 0.05552
Best fitness: 0.30114 - size: (30, 50) - species 156 - id 64592
Average adjusted fitness: 0.110
Mean genetic distance 2.965, standard deviation 0.523
Population of 201 members in 14 species
Total extinctions: 0
Generation time: 4.653 sec (4.573 average)

 ****** Running generation 386 ****** 

Population's average fitness: 0.16758 stdev: 0.05982
Best fitness: 0.30114 - size: (30, 50) - species 156 - id 64592
Average adjusted fitness: 0.153
Mean genetic distance 2.949, standard deviation 0.528
Population of 198 members in 14 species
Total extinctions: 0
Generation time: 4.974 sec (4.598 averag

Population's average fitness: 0.17433 stdev: 0.06153
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.144
Mean genetic distance 2.965, standard deviation 0.508
Population of 204 members in 12 species
Total extinctions: 0
Generation time: 4.829 sec (5.087 average)

 ****** Running generation 409 ****** 

Population's average fitness: 0.17115 stdev: 0.05891
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.138
Mean genetic distance 3.003, standard deviation 0.493
Population of 199 members in 13 species
Total extinctions: 0
Generation time: 4.934 sec (5.103 average)

 ****** Running generation 410 ****** 

Population's average fitness: 0.17219 stdev: 0.05947
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.130
Mean genetic distance 2.994, standard deviation 0.490
Population of 202 members in 13 species
Total extinctions: 0
Generation time: 4.843 sec (5.120 averag

Population's average fitness: 0.17523 stdev: 0.06364
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.125
Mean genetic distance 2.902, standard deviation 0.481
Population of 195 members in 16 species
Total extinctions: 0
Generation time: 4.480 sec (4.407 average)

 ****** Running generation 433 ****** 

Population's average fitness: 0.17422 stdev: 0.06088
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.157
Mean genetic distance 2.863, standard deviation 0.518
Population of 201 members in 14 species
Total extinctions: 0
Generation time: 4.559 sec (4.430 average)

 ****** Running generation 434 ****** 

Population's average fitness: 0.16995 stdev: 0.06132
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.107
Mean genetic distance 2.866, standard deviation 0.510
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 4.909 sec (4.494 averag

Population's average fitness: 0.16990 stdev: 0.05884
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.130
Mean genetic distance 2.707, standard deviation 0.558
Population of 197 members in 8 species
Total extinctions: 0
Generation time: 4.198 sec (4.452 average)

 ****** Running generation 457 ****** 

Population's average fitness: 0.16777 stdev: 0.05922
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.103
Mean genetic distance 2.734, standard deviation 0.542
Population of 199 members in 8 species
Total extinctions: 0
Generation time: 4.493 sec (4.448 average)

 ****** Running generation 458 ****** 

Population's average fitness: 0.16550 stdev: 0.06116
Best fitness: 0.30114 - size: (30, 46) - species 169 - id 65470
Average adjusted fitness: 0.115
Mean genetic distance 2.607, standard deviation 0.488
Population of 200 members in 7 species
Total extinctions: 0
Generation time: 4.152 sec (4.422 average)


Population's average fitness: 0.16484 stdev: 0.05723
Best fitness: 0.30682 - size: (40, 64) - species 192 - id 81342
Average adjusted fitness: 0.122
Mean genetic distance 2.734, standard deviation 0.449
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 4.631 sec (4.952 average)

 ****** Running generation 481 ****** 

Population's average fitness: 0.16427 stdev: 0.05727
Best fitness: 0.30682 - size: (40, 64) - species 192 - id 81342
Average adjusted fitness: 0.144
Mean genetic distance 2.745, standard deviation 0.431
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 4.762 sec (4.974 average)

 ****** Running generation 482 ****** 

Population's average fitness: 0.16168 stdev: 0.05488
Best fitness: 0.30682 - size: (40, 64) - species 192 - id 81342
Average adjusted fitness: 0.100
Mean genetic distance 2.722, standard deviation 0.486
Population of 203 members in 13 species
Total extinctions: 0
Generation time: 4.557 sec (4.948 averag

In [69]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-vowel11")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [70]:
testsamplesize = 462
outputs = []
for xi in test_x_inputs:
    output = winner_net.activate(xi)
    outputs.append(output)

px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 11), axis=1)
# the index of maximum in each line
pred_outputs = np.argmax(px_outputs, axis = 1)
real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

acc = np.sum(pred_outputs == real_outputs)/testsamplesize
print("Accuracy: {}".format(acc))

Accuracy: 0.20562770562770563
